## Logistic Regression in Theano

In [3]:
import theano
import theano.tensor as T
import numpy as np

In [4]:
rng = np.random

In [5]:
N = 400                                   # training sample size
feats = 784                               # number of input variables

In [7]:
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

In [8]:
# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

In [21]:
# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

In [22]:
# initialize the bias term
b = theano.shared(0., name="b")

In [23]:
print("Initial weights:")
print(w.get_value())[:5]
print(b.get_value())

Initial weights:
[ 1.10165219  0.05109273  0.31277315 -1.42387098  1.23499514]
0.0


In [14]:
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

In [15]:
# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

In [17]:
# Train
for i in range(training_steps):
    predicted, loss_function = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[ -3.87701542e-02  -1.25615698e-01  -1.27001865e-01   8.70123756e-02
   2.33038200e-02   4.61146617e-02  -3.30625873e-02  -4.19552391e-02
  -1.43718707e-01  -1.40840435e-03  -1.00191104e-01  -7.34545748e-02
   7.53501156e-02  -1.02120412e-01  -6.12477639e-03  -1.60967413e-01
   2.78665275e-02  -1.25072226e-01  -1.27091416e-01   7.02829621e-02
  -1.31657581e-01   3.68352788e-02   8.80679202e-02  -6.44575468e-02
   7.45724853e-02   3.75750111e-02   1.73887680e-02  -2.11387768e-01
  -5.52259970e-02  -1.22177764e-01  -6.60097280e-03  -1.36484558e-01
  -6.73816622e-02  -4.91640061e-02  -1.03789569e-01  -3.96336023e-02
  -2.09692717e-02  -3.00153944e-02  -1.63042073e-01  -7.10332955e-02
   8.62960161e-02   3.42151719e-03   9.17759421e-03   7.36822822e-02
   7.24379875e-03   4.42635356e-02  -2.40747544e-02  -1.67864611e-02
  -3.27298526e-02   1.01414118e-01   5.03150416e-02  -1.74057269e-01
   8.17638848e-02  -4.95204061e-02   5.46743132e-02   5.97535221e-03
  -3.69669414e-02  -2

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
accuracy_score(D[1], predict(D[0]))

1.0